## Overview of Functions

Let us get an overview of different functions that are available to process data in columns.

* While Data Frame APIs work on the Data Frame, at times we might want to apply functions on column values.
* Functions to process column values are available under `pyspark.sql.functions`. These are typically used in select or withColumn on top of Data Frame.
* There are approximately 300 pre-defined functions available for us.
* Some of the important functions can be broadly categorized into String Manipulation, Date Manipulation, Numeric Functions and Aggregate Functions.
* String Manipulation Functions
  * Concatenating Strings - `concat`
  * Getting Length - `length`
  * Trimming Strings - `trim`,` rtrim`, `ltrim`
  * Padding Strings - `lpad`, `rpad`
  * Extracting Strings - `split`, `substring`
* Date Manipulation Functions
  * Date Arithmetic - `date_add`, `date_sub`, `datediff`, `add_months`
  * Date Extraction - `dayofmonth`, `month`, `year`
  * Get beginning period - `trunc`, `date_trunc`
* Numeric Functions - `abs`, `greatest`
* Aggregate Functions - `sum`, `min`, `max`

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.

In [1]:
from pyspark.sql import SparkSession

import getpass
username = getpass.getuser()

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    appName(f'{username} | Python - Data Processing - Overview'). \
    master('yarn'). \
    getOrCreate()

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

In [2]:
employees = [(1, "Scott", "Tiger", 1000.0, "united states"),
             (2, "Henry", "Ford", 1250.0, "India"),
             (3, "Nick", "Junior", 750.0, "united KINGDOM"),
             (4, "Bill", "Gomes", 1500.0, "AUSTRALIA")
            ]

In [3]:
employeesDF = spark. \
    createDataFrame(employees,
                    schema="""employee_id INT, first_name STRING, 
                    last_name STRING, salary FLOAT, nationality STRING"""
                   )

In [4]:
employeesDF.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- salary: float (nullable = true)
 |-- nationality: string (nullable = true)



In [5]:
employeesDF.show()

+-----------+----------+---------+------+--------------+
|employee_id|first_name|last_name|salary|   nationality|
+-----------+----------+---------+------+--------------+
|          1|     Scott|    Tiger|1000.0| united states|
|          2|     Henry|     Ford|1250.0|         India|
|          3|      Nick|   Junior| 750.0|united KINGDOM|
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|
+-----------+----------+---------+------+--------------+



### Tasks
Let us perform a task to understand how functions are typically used.

* Project full name by concatenating first name and last name along with other fields excluding first name and last name.

In [8]:
from pyspark.sql.functions import lit, concat

In [9]:
employeesDF. \
    withColumn("full_name", concat("first_name", ", ", "last_name")). \
    drop("first_name", "last_name"). \
    show()

AnalysisException: "cannot resolve '`, `' given input columns: [nationality, employee_id, last_name, salary, first_name];;\n'Project [employee_id#0, first_name#1, last_name#2, salary#3, nationality#4, concat(first_name#1, ', , last_name#2) AS full_name#27]\n+- LogicalRDD [employee_id#0, first_name#1, last_name#2, salary#3, nationality#4], false\n"

In [10]:
employeesDF. \
    withColumn("full_name", concat("first_name", lit(", "), "last_name")). \
    drop("first_name", "last_name"). \
    show()

+-----------+------+--------------+------------+
|employee_id|salary|   nationality|   full_name|
+-----------+------+--------------+------------+
|          1|1000.0| united states|Scott, Tiger|
|          2|1250.0|         India| Henry, Ford|
|          3| 750.0|united KINGDOM|Nick, Junior|
|          4|1500.0|     AUSTRALIA| Bill, Gomes|
+-----------+------+--------------+------------+



In [11]:
employeesDF. \
    select("employee_id",
           concat("first_name", lit(", "), "last_name"),
           "salary",
           "nationality"
          ). \
    show()

+-----------+---------------------------------+------+--------------+
|employee_id|concat(first_name, , , last_name)|salary|   nationality|
+-----------+---------------------------------+------+--------------+
|          1|                     Scott, Tiger|1000.0| united states|
|          2|                      Henry, Ford|1250.0|         India|
|          3|                     Nick, Junior| 750.0|united KINGDOM|
|          4|                      Bill, Gomes|1500.0|     AUSTRALIA|
+-----------+---------------------------------+------+--------------+



In [12]:
employeesDF. \
    select("employee_id",
           concat("first_name", lit(", "), "last_name").alias("full_name"),
           "salary",
           "nationality"
          ). \
    show()

+-----------+------------+------+--------------+
|employee_id|   full_name|salary|   nationality|
+-----------+------------+------+--------------+
|          1|Scott, Tiger|1000.0| united states|
|          2| Henry, Ford|1250.0|         India|
|          3|Nick, Junior| 750.0|united KINGDOM|
|          4| Bill, Gomes|1500.0|     AUSTRALIA|
+-----------+------------+------+--------------+



In [13]:
employeesDF. \
    selectExpr("employee_id",
               "concat(first_name, ', ', last_name) AS full_name",
               "salary",
               "nationality"
              ). \
    show()

+-----------+------------+------+--------------+
|employee_id|   full_name|salary|   nationality|
+-----------+------------+------+--------------+
|          1|Scott, Tiger|1000.0| united states|
|          2| Henry, Ford|1250.0|         India|
|          3|Nick, Junior| 750.0|united KINGDOM|
|          4| Bill, Gomes|1500.0|     AUSTRALIA|
+-----------+------------+------+--------------+



**We will explore most of the functions as we get into the data processing at a later point in time**